In [1]:
import core
import pandas as pd
from datetime import datetime
import ast

In [2]:
current_date = "2025-07-23"
datetime_obj = pd.to_datetime(current_date)

In [3]:
core.MonteCarloSim(match_id=4640)

Simulating Ceará vs Mirassol


Simulations (1 worker):   0%|          | 0/1 [00:00<?, ?it/s]


0.10416172227514542 0.1115270837751008


AttributeError: 'MonteCarloSim' object has no attribute 'get_time_segment'